In [ ]:
# from tqdm.auto import tqdm
# from time import sleep

# for i in (ep_bar:= tqdm(range(4), desc='1st loop')):
#     ep_bar.set_description(f'Epoch {i}')
#     for j in tqdm(range(5), desc='2nd loop', leave= False):
#         for k in tqdm(range(50), desc='3rd loop', leave=False):
#             sleep(0.01)

In [1]:
# raw = input("Test: ")

In [ ]:
# for i, j in enumerate([1,2,3,4,5,6,7,8]):
#     print(i,j)

In [147]:
import patcher

In [3]:
import torch
from torchvision import models

In [4]:
model = models.resnet50().cuda()

In [9]:
from spoof_gen.models.backbone.iresnet import iresnet

imodel = iresnet('50').cuda()

In [12]:
imodel(torch.rand((N,3,256,256)).cuda())

torch.Size([1, 512, 16, 16])


RuntimeError: mat1 and mat2 shapes cannot be multiplied (1x131072 and 25088x512)

In [ ]:
Encoder('resnet-50').backbone

In [ ]:
model

In [6]:
model(x).shape

torch.Size([1, 1000])

In [152]:
def condition(layer, name):
    return hasattr(layer, "stride") and name == "conv1"

In [153]:
def patch_fn(layer):
    layer.stride = (1, 1)
    return layer

In [ ]:
patcher.patch_net(model, condition=condition, patch=patch_fn)

In [ ]:
# from torchsummary import summary
# summary(model,(3,256,256))

In [1]:
import torch
from spoof_gen.models.backbone.upblock import build_decoder_unet
from spoof_gen.models.backbone.mobilenet import mobilenet
from spoof_gen.models.backbone.resnet import resnet
from spoof_gen.models.backbone.revnet import revnet
from spoof_gen.models.generator import Encoder, Decoder, Generator
from spoof_gen.models.conditional_generator import ConditionGenerator
from spoof_gen.models.critic import Critic
# decoder = build_decoder_unet(skip_z= True)

N = 1
x = torch.rand((N,3,256,256)).cuda()
z = torch.rand((N,4,32,32)).cuda()

In [2]:
import torch 
import torch.nn as nn
import numpy as np
import math
import torch.nn.functional as F

def contrast_conv(input : torch.Tensor):
    ''' compute contrast depth in both of (out, label) '''
    '''
        input  Nx3x256x256
        output Nx(8x3)x32x32
    '''

    device = input.device

    N,C,H,W = input.shape

    input = input.view(N*C,H,W)

    kernel_filter_list = [
        [[1, 0, 0], [0, -1, 0], [0, 0, 0]], [[0, 1, 0], [0, -1, 0], [0, 0, 0]], [[0, 0, 1], [0, -1, 0], [0, 0, 0]],
        [[0, 0, 0], [1, -1, 0], [0, 0, 0]], [[0, 0, 0], [0, -1, 1], [0, 0, 0]],
        [[0, 0, 0], [0, -1, 0], [1, 0, 0]], [[0, 0, 0], [0, -1, 0], [0, 1, 0]], [[0, 0, 0], [0, -1, 0], [0, 0, 1]]]

    kernel_filter = np.array(kernel_filter_list, float)

    kernel_filter = torch.from_numpy(
        kernel_filter.astype(np.float)).float().to(device)
    # weights (in_channel, out_channel, kernel, kernel)
    kernel_filter = kernel_filter.unsqueeze(dim=1)


    input = input.unsqueeze(dim=1).expand(
        input.shape[0], 8, input.shape[1], input.shape[2])

    contrast_img = F.conv2d(
        input, weight=kernel_filter, groups=8)  # depthwise conv

    contrast_img = contrast_img.view(N,8,C,H-2,W-2)
    return contrast_img

# Pearson range [-1, 1] so if < 0, abs|loss| ; if >0, 1- loss
class Contrast_loss:
    def __init__(self):
        pass

    def __call__(self, out, label):
        '''
        compute contrast depth in both of (out, label),
        then get the loss of them
        tf.atrous_convd match tf-versions: 1.4
        '''

        contrast_out = contrast_conv(out)
        
        contrast_label = contrast_conv(label)

        criterion_MSE = nn.MSELoss()

        loss = criterion_MSE(contrast_out, contrast_label)
        #loss = torch.pow(contrast_out - contrast_label, 2)
        #loss = torch.mean(loss)

        return loss
    


In [3]:
from spoof_gen.data_utils import StandardDataset
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
train_data = StandardDataset('Data/train_img', transform= None)
train_loader = DataLoader(train_data,2)

# loss_cdl = Contrast_loss()

# idx = np.random.randint(0,len(train_data))
# idx2 = np.random.randint(0,len(train_data))

# # loss_cdl(train_data[idx][1].unsqueeze(0).unsqueeze(0),train_data[idx2][1].unsqueeze(0).unsqueeze(0))
# nn.MSELoss()(train_data[idx][1].unsqueeze(0),train_data[idx2][1].unsqueeze(0))

# for img in contrast_conv(train_data[idx][0].unsqueeze(0)).squeeze(0):
#     plt.imshow(img.permute(1,2,0).cpu().numpy())
#     plt.show()

In [4]:
con_gen = ConditionGenerator(init_from='checkpoints/conditional_generator/conditonal_generator_mobilenetv3_last.pth').cuda()
encoder = Encoder('iresnet-50').cuda()
decoder = Decoder('revnet-50').cuda()
gen = Generator(encoder= encoder, decoder= decoder, con_gen= con_gen)
crit = Critic().cuda()

All states are loaded successfully!


In [5]:
inputs, map_x, label, _ = next(iter(train_loader))
inputs = inputs.cuda()
map_x = map_x.cuda()

In [6]:
map_x = con_gen(inputs,label)
crit(inputs,map_x)

tensor([[0.2951],
        [0.1484]], device='cuda:0', grad_fn=<AddmmBackward0>)

In [6]:
encoder.load_pretrain_backbone('checkpoints/backbone_arcface_50.pth')

['fc.weight',
 'fc.bias',
 'features.weight',
 'features.bias',
 'features.running_mean',
 'features.running_var']

In [47]:
import torch
import torch.nn as nn

class SimpleVAE(nn.Module):
    def __init__(self, in_dim = 256) -> None:
        super().__init__()
        self.encoder = nn.Sequential(nn.Linear(in_dim, 128),
                                     nn.ReLU(inplace= True),
                                     nn.Linear(128, 64),
                                     nn.ReLU(inplace= True),
                                     nn.Linear(64, 32),
                                     nn.ReLU(inplace= True),
                                     nn.Linear(32, 16 *2))
        
        self.decoder = nn.Sequential(
                                     nn.Linear(16, 32),
                                     nn.ReLU(inplace= True),
                                     nn.Linear(32, 64),
                                     nn.ReLU(inplace= True),
                                     nn.Linear(64, 128),
                                     nn.ReLU(inplace= True),
                                     nn.Linear(128, 256))
    
    def forward(self, x):
        mu, log_var = torch.split(self.encoder(x), split_size_or_sections= 16, dim= 1)
        z = mu + torch.rand_like(log_var) * log_var
        out = self.decoder(z)

        return out
    
class SimpleGAN(nn.Module):
    def __init__(self, in_dim = 256) -> None:
        super().__init__()
        self.net = nn.Sequential(nn.Linear(in_dim, 128),
                                 nn.ReLU(inplace= True),
                                 nn.Linear(128, 64),
                                 nn.ReLU(inplace= True),
                                 nn.Linear(64, 32),
                                 nn.ReLU(inplace= True),
                                 nn.Linear(32, 1),
                                 nn.Sigmoid())
    
    def forward(self, x):
        return self.net(x)
    
    def get_grad(self):
        grad = []
        for param in self.parameters():
            grad.append(torch.tensor(param.grad))
        self.grad_ = grad
        del grad
    
    def load_grad(self):
        for param in self.parameters():
            param.grad.copy_(self.grad_.pop(0))

        del self.grad_


gen = SimpleVAE()
dis = SimpleGAN()

g_opt = torch.optim.Adam(gen.parameters(),lr = 0.0001)
d_opt = torch.optim.Adam(dis.parameters(),lr = 0.0001)

x = torch.rand(4,256)
out  :torch.Tensor = gen(x)

d_real = dis(x)
d_gen  = dis(out)
loss = (d_gen - d_real).mean()
# loss = (((x - out)**2)/2).mean()

loss.backward()


In [49]:
x = torch.rand(4,256)
out  :torch.Tensor = gen(x)

d_real = dis(x)
d_gen  = dis(out)
loss = (d_gen - d_real).mean()
# loss = (((x - out)**2)/2).mean()

loss.backward()

In [23]:
# net.eval()
g_opt.step()

In [48]:
# dis.get_grad()
dis.grad_[0]

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_30856\3275356399.py:49: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  grad.append(torch.tensor(param.grad))


tensor([[ 3.7181e-04,  5.5979e-04,  7.6311e-04,  ...,  6.3562e-04,
          5.5945e-04,  3.5486e-04],
        [-8.5978e-05, -2.9133e-05, -2.6214e-05,  ..., -7.6220e-05,
         -1.0989e-04, -6.6789e-06],
        [-1.2898e-04, -1.3837e-04, -1.3400e-04,  ..., -1.6622e-04,
         -1.9605e-04, -8.0626e-05],
        ...,
        [ 3.9026e-04,  9.3475e-04,  7.9567e-04,  ...,  5.6885e-04,
          5.1848e-04,  9.0690e-04],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        [-3.2699e-04, -3.1702e-04, -1.7325e-04,  ..., -2.2452e-04,
         -1.9004e-04, -3.3440e-04]])

In [27]:
d_opt.step()

In [52]:
dis.net[0].weight.grad

tensor([[ 3.7181e-04,  5.5979e-04,  7.6311e-04,  ...,  6.3562e-04,
          5.5945e-04,  3.5486e-04],
        [-8.5978e-05, -2.9133e-05, -2.6214e-05,  ..., -7.6220e-05,
         -1.0989e-04, -6.6789e-06],
        [-1.2898e-04, -1.3837e-04, -1.3400e-04,  ..., -1.6622e-04,
         -1.9605e-04, -8.0626e-05],
        ...,
        [ 3.9026e-04,  9.3475e-04,  7.9567e-04,  ...,  5.6885e-04,
          5.1848e-04,  9.0690e-04],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        [-3.2699e-04, -3.1702e-04, -1.7325e-04,  ..., -2.2452e-04,
         -1.9004e-04, -3.3440e-04]])

In [51]:
dis.load_grad()

In [29]:
def plus_one(a):
    a += 1

x = 5
plus_one(x)
x

5

In [17]:
encoder.backbone._modules['fc']

False

In [9]:
for i in range(1):
    gen_resp = gen(x,1)

In [4]:
map_x = con_gen(x,1)
crit(x,map_x)

torch.Size([1, 1, 256, 256])
torch.Size([1, 3, 256, 256])


tensor([[0.0449]], device='cuda:0', grad_fn=<AddmmBackward0>)

In [10]:
crit(gen_resp[0][0],gen_resp[1][0])

ValueError: Input and output must have the same number of spatial dimensions, but got input with spatial dimensions of [32] and output size of (256, 256). Please provide input tensor in (N, C, d1, d2, ...,dK) format and output size in (o1, o2, ...,oK) format.

In [ ]:
!pip install git+https://github.com/minh-nguyenhoang/SpoofGen

In [2]:
import spoof_gen